In [21]:
import pickle
import pandas as pd

In [68]:
pretrain = pd.read_pickle('user_pretrain_2.pk')
users_partition = pd.read_pickle('C_U_type-1_num-10.pk')

In [69]:
pretrain.shape

(6040, 64)

In [70]:
len(users_partition)

10

I went to the Movielens page, downloaded the files and read the readme.

"These files contain 1,000,209 anonymous ratings of approximately 3,900 movies made by 6,040 MovieLens users who joined MovieLens in 2000."

This tells me that pretrain is (people, y), but idk what y is. users_partition are split across 10 partitions, so it's not that.

In [82]:
users = pd.read_fwf('users.dat',header=None)

In [83]:
users.head()

,0
0,1::F::1::10::48067
1,2::M::56::16::70072
2,3::M::25::15::55117
3,4::M::45::7::02460
4,5::M::25::20::55455


Readme tells me that the file is in this format= UserID::Gender::Age::Occupation::Zip-code

In [84]:
users = users[0].str.split('::',expand=True)
users.columns=['UserID','Gender','Age','Occupation','Zipcode']

In [89]:
# users.set_index('UserID').to_csv('users.csv', index=None) # Don't use this because they switched it to 0 indexing
users[['Gender','Age','Occupation','Zipcode']].to_csv('users.csv')

We want a dataframe that is users X partitions. Let's create one for each of the files partitioned by users.

In [142]:
user_files = ['C_U_type-1_num-10.pk','C_U_type-2_num-10.pk','C_U_type-3_num-10.pk']

for file_index, file in enumerate(user_files):
    # Open the paritioned user file
    users_partition = pd.read_pickle(file)

    # Make a dummy dataframe rows=persons, cols=userID + partitions
    users_type = pd.DataFrame(list(range(0,6040)),columns=['UserID',])
    cols = ['p0','p1','p2','p3','p4','p5','p6','p7','p8','p9']
    for col in cols:
        users_type[col] = 0
    
    # Populate cols with 1 if userID is in that partition
    for idx, partition in enumerate(users_partition):
        users_type.iloc[partition, idx+1] = 1
        
    out_index = file_index +1
    users_type.to_csv(f'C_U_type-{out_index}_num-10.csv',index=None)